# 데이터 구조 이해 & SQL 스키마 설계
- JSON 데이터 로드 및 컬럼 확인 (pandas)
- 결측치·중복 데이터 확인/정제
- SQL DB 스키마 설계 (Supplements, Ingredients, Supplement_Ingredients, Symptoms_Ingredients)• 증상–성분 매핑 테이블 초안 작성

- json 파일 데이터 개수: 1241개, 23열 ( )

In [ ]:
import pandas as pd
import numpy as np

In [ ]:

# 1) 로드
fp = '/Users/gim-yujin/Desktop/pjt_personal_agent/영양소 데이터/iherb_data_uk_data_2022_12.json'
df = pd.read_json(fp, orient='records')   # 파일이 리스트 of dicts 여야 정상
# 2) 전체 컬럼 확인
print(df.shape)
print(df.columns.tolist())
# 3) 샘플 확인
display(df.head())
# 4) 기본 타입 정리
df['scraped_at'] = pd.to_datetime(df['scraped_at'], dayfirst=True, errors='coerce')

In [ ]:
print("##컬럼명 목록")
print(df.columns)
print("-" * 50)
# 컬럼별 결측치 개수 확인 
print("##컬럼별 결측치 개수 확인(공백 문자열이 결측치로 합산이 안되어 모두 0으로 표기됨)")
print(df.isnull().sum())

### 결측치 확인
- 결측값이 존재하는 컬럼 및 개수
- Category 2            4
- Category 3          869
- ingredients          61
- Supplement Facts    530 (추후에 성분을 참고하여 채워 넣을 예정, 보충정보(영양성분))

In [ ]:
# (선택) 모든 컬럼에 대해 한 번에 적용할 수도 있습니다.
df.replace(r'^\s*$', np.nan, regex=True, inplace=True)
print("\n## 전체 데이터의 실제 결측치 수")
print(df.isnull().sum())

In [ ]:
#  각 컬럼의 결측치 개수를 계산
missing_values = df.isnull().sum()
#  결측치 개수가 0보다 큰 컬럼들만 필터링하여 출력합니다.
columns_with_missing_values = missing_values[missing_values > 0]

print("## 결측값이 존재하는 컬럼 및 개수")
if columns_with_missing_values.empty:
    print("모든 컬럼의 데이터가 채워져 있습니다.")
else:
    print(columns_with_missing_values)

In [ ]:
#결측값이 존재하는 컬럼 선택 출력 확인 

df_missing = df[columns_with_missing_values.index]
print(df_missing)

In [ ]:
df[columns_with_missing_values.index]

### 중복 제거 

- unique_id 기준(0)
- Pid, title 기준(0)
- 중복되는 아이템은 없음


In [ ]:
# 중복 제거 (uniqe_id 기준으로는 중복 없음.)
df = df.drop_duplicates(subset=['uniq_id'])  
# uniq_id가 있으면 안전
df.drop_duplicates(subset=['Pid','Title'])
print(df)

In [ ]:
df.drop_duplicates(subset=['Pid','Title']) #여기도 중복은 없는 것으로 확인 

In [ ]:
df['Price'] = df['Price'].astype(str)
# price >> 문자열로 공백/ 쉼표 제고후 float
df['Price'] = df['Price'].str.replace(',', '').str.strip()
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')


### 불필요한 레코드 필터링 
- 카테고리 1, 2 기준으로 키워드 필터링 한 결과, 해당 데이터셋에서 영양제와 관련된 상품 개수는 1221->325 개로 감소함.


In [ ]:
## 화장품/ 샴푸/ 식품/ 베이비 용품 등 필터링 
non_supp_cats = ['Shampoo','Foundation','Face Wash','Utensils','Diapers']  # 예시
df = df[~df['Category 2'].isin(non_supp_cats)]
df.count

In [ ]:
# 'Category 1' 컬럼의 모든 고유값 출력
print("Category 1의 고유값:", df['Category 1'].unique())

# 'Category 2' 컬럼의 모든 고유값 출력
print("Category 2의 고유값:", df['Category 2'].unique())

# 'Category 3' 컬럼의 모든 고유값 출력
print("Category 3의 고유값:", df['Category 3'].unique())

In [ ]:
## 화이트리스트 키워드(supp_keywords)

supp_keywords = [
    # 비타민·미네랄
    'vitamin', 'multivitamin', 'multimineral', 'b1', 'b2', 'b3', 'b6',
    'b12', 'c', 'd', 'e', 'k', 'folic acid', 'niacin', 'biotin',
    'calcium', 'magnesium', 'zinc', 'iron', 'selenium', 'potassium',
    'iodine', 'trace minerals',

    # 오메가 & 필수지방산
    'omega', 'fish oil', 'krill oil', 'cod liver oil',
    'efa', 'dha', 'epa',

    # 허브·식물 추출물
    'herb', 'herbal', 'ashwagandha', 'ginseng', 'echinacea', 'turmeric',
    'curcumin', 'milk thistle', 'rhodiola', 'elderberry', 'boswellia',
    'sambucus', 'hawthorn', 'garlic', 'ginger', 'licorice', 'oregano',
    'passion flower', 'valerian', 'chamomile', 'nettle', 'schisandra',
    'astragalus',

    # 아미노산·단백질
    'amino', 'amino acid', 'l-',   # L-Arginine, L-Tyrosine 등 앞에 L-이 붙음
    'protein', 'collagen', 'peptide',

    # 프로바이오틱/소화
    'probiotic', 'prebiotic', 'lactobacillus', 'bifidus',
    'digestive enzymes', 'enzyme',

    # 항산화·기타 보조성분
    'coq10', 'ubiquinol', 'alpha lipoic acid', 'resveratrol',
    'pycnogenol', 'glutathione', 'chlorophyll', 'spirulina',
    'chlorella', 'maca', 'bee pollen', 'royal jelly',

    # 특수 목적 포뮬러
    'immune', 'immune support', 'energy formula', 'sleep formula',
    'cognitive', 'memory', 'joint', 'bone', 'liver', 'thyroid',
    'blood support', 'heart support', 'detox', 'women\'s health',
    'men\'s health', 'prenatal', 'post-natal',
    'sports supplement', 'workout', 'weight management', 'fat burner',

    # 형태·일반명
    'supplement', 'dietary', 'nutrition', 'nutrient',
    'superfood', 'greens', 'superfood blend'
]


In [ ]:
import re

def is_supplement_row(row):
    cats = " ".join([
        str(row.get('Category 2', '')).lower(),
        str(row.get('Category 3', '')).lower()
    ])
    return any(re.search(rf"\b{k}\b", cats) for k in supp_keywords)

df_supp = df[df.apply(is_supplement_row, axis=1)].copy()

print(f"필터 전 {len(df)} → 필터 후 {len(df_supp)}")

### 제대로 필터링 되었는지 확인 작업
- 멀티비타민과, 비타민 구분하여 놓았는지 

In [ ]:
import re

# 1️⃣ 화이트리스트 기반 영양제 필터
def is_supplement_row(row):
    cats = " ".join([
        str(row.get('Category 2', '')).lower(),
        str(row.get('Category 3', '')).lower()
    ])
    return any(re.search(rf"\b{k}\b", cats) for k in supp_keywords)

df_supp = df[df.apply(is_supplement_row, axis=1)].copy()

# 2️⃣ 블랙리스트 제거 (현재는 K-Beauty 하나지만 확장 가능)
black_keywords = ['k-beauty']
def not_blacklisted(row):
    cats = " ".join([
        str(row.get('Category 1', '')).lower(),
        str(row.get('Category 2', '')).lower(),
        str(row.get('Category 3', '')).lower()
    ])
    return not any(bk in cats for bk in black_keywords)

df_supp = df_supp[df_supp.apply(not_blacklisted, axis=1)].copy()

print(f"최종 필터 후 행 수 : {len(df_supp)}")

In [ ]:
# 3️⃣ 무작위 100건 추출 (중복 없이)
sample_100 = df_supp.sample(n=100, random_state=42)  # random_state는 재현성

# 4️⃣ 검토에 유용한 컬럼만 보기
cols_to_check = ['Title', 'Category 1', 'Category 2', 'Category 3', 'Description']
print(sample_100[cols_to_check].to_string(index=False))



### “파싱(parsing)”: 문자열(예: Supplement Facts 텍스트) 안에서 우리가 원하는 정보(성분명, 용량, 단위 등)를 규칙적으로 뽑아내는 작업

- 1.	parse_supplement_facts()
→ 텍스트를 줄 단위로 읽고 정규식으로 [성분, 수치, 단위] 추출.
- 2.	parse_and_flag()
→ 전체 DataFrame에 적용, parsed_ingredients와 parse_error 컬럼 추가.
- 3.	수동 검토
→ parse_error=True인 레코드만 CSV로 내보내어 직접 확인·수정.
--- 
- df_supp_checked

- parsed_ingredients: 파싱 성공 시 [{'name':…, 'amount':…, 'unit':…}, …] 리스트

- parse_error: True(실패) / False(성공)

- supplement_parse_errors.csv

사람이 직접 살펴보고 정규식 보완이나 데이터 수동 입력이 필요한 상품 목록.


In [87]:
keyword_pattern = r'(?i)' + '|'.join([re.escape(k) for k in supp_keywords])

In [88]:
def parse_supplement_facts(text):
    """
    Supplement Facts 문자열에서
    [성분명, 수치, 단위] 추출
    """
    results = []
    if not isinstance(text, str) or not text.strip():
        return results  # 빈 값이면 바로 실패
    
    # 예) "Vitamin C 500 mg", "Magnesium (as oxide) 250 mg"
    pattern = r'([A-Za-z0-9\-\(\) /]+?)\s+([\d.,]+)\s*(mg|mcg|µg|g|iu|IU)'
    
    for line in text.splitlines():
        m = re.search(pattern, line)
        if m:
            name = m.group(1).strip()
            amount = float(m.group(2).replace(',', ''))
            unit = m.group(3).lower()
            results.append({
                'name': name,
                'amount': amount,
                'unit': unit,
                'raw_line': line.strip()
            })
    return results

In [89]:
def parse_and_flag_supp(df_supp):
    parsed_results = []
    parse_error_flags = []

    for text in df_supp['Supplement Facts']:
        parsed = parse_supplement_facts(text)
        parsed_results.append(parsed)
        # 파싱 결과가 없으면 True → 실패
        parse_error_flags.append(len(parsed) == 0)

    df_supp_checked = df_supp.copy()
    df_supp_checked['parsed_ingredients'] = parsed_results
    df_supp_checked['parse_error'] = parse_error_flags
    return df_supp_checked

# ✅ 실행
df_supp_checked = parse_and_flag_supp(df_supp)


In [90]:
len(df_supp_checked)

325

In [91]:

# 파싱 실패 데이터만 추출
df_supp_errors = df_supp_checked[df_supp_checked['parse_error']]

# 주요 컬럼만 저장
cols_to_review = ['Title', 'Category 2', 'Category 3', 'Supplement Facts']
df_supp_errors[cols_to_review].to_csv('supplement_parse_errors.csv', index=False)

print(f"파싱 실패 레코드 수: {len(df_supp_errors)}")
print("수동 검토 파일: supplement_parse_errors.csv 저장 완료!")


파싱 실패 레코드 수: 198
수동 검토 파일: supplement_parse_errors.csv 저장 완료!


In [92]:
df_supp_errors = df_supp_checked[df_supp_checked['parse_error']]
print("파싱 실패 레코드 수:", len(df_supp_errors))
df_supp_errors[['Title','Category 2','Category 3','Supplement Facts']].head(10)

파싱 실패 레코드 수: 198


,Title,Category 2,Category 3,Supplement Facts
0,"MediNatura, WellMind Calming Day/Night, 100 Ta...",Homeopathy Formulas,Cognitive & Memory Formulas,NaN
3,"Natrol, B-Complex, Fast Dissolve, Coconut, 90 ...",Vitamin B Complex,NaN,Supplement FactsServing Size: 1 TabletServings...
24,"NOW Foods, Glycine, Pure Powder, 1 lb (454 g)",L-Glycine,NaN,\n\nSupplement Facts\n\n\nServing Size: 3/4 Le...
44,"Nature's Bounty, Selenium, 200 mcg, 100 Tablets",Selenium,NaN,Supplement FactsServing Size: 1 TabletAmount P...
45,"Dr. Mercola, Zinc plus Selenium, 30 Capsules",Zinc,Selenium,Supplement FactsServing Size: 1 CapsuleServing...
48,"Zand, Echinacea Zinc, Very Cherry, 80 Throat L...",Zinc,Sore Throat & Cough Lozenges,Supplement FactsServing Size: 1 lozenge (3.8 g...
56,"Harney & Sons, Paris Tea, 1 lb",Black Tea,Herbal Tea,NaN
58,"Crystal Star, Liver Renew, 90 Vegetarian Capsules",Liver Formulas,NaN,Supplement FactsServing Size: 2 capsulesServin...
59,"Superior Source, Methylcobalamin B-12, B-6 & F...",Vitamin B,NaN,Supplement FactsServing Size: 1 MicroLingual® ...
61,"Life Extension, Standardized Cistanche, 30 Veg...",Herbs,Immune Formulas,Supplement FactsServing Size: 1 Vegetarian Cap...


### 파싱 실패 레코드 처리 
- 